# Praca domowa 4
#### Jakub Lis

## Wczytanie pakietów

In [21]:
# pip install dalex -U
import dalex as dx
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.datasets import load_wine
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

## Wczytanie zbiorów danych

In [22]:
data = dx.datasets.load_apartments()

In [23]:
data

,m2_price,construction_year,surface,floor,no_rooms,district
1,5897,1953,25,3,1,Srodmiescie
2,1818,1992,143,9,5,Bielany
3,3643,1937,56,1,2,Praga
4,3517,1995,93,7,3,Ochota
5,3013,1992,144,6,5,Mokotow
...,...,...,...,...,...,...
996,6355,1921,44,2,2,Srodmiescie
997,3422,1921,48,10,2,Bemowo
998,3098,1980,85,3,3,Bemowo
999,4192,1942,36,7,1,Zoliborz


Wybrany przeze mnie drugi zbiór danych zawiera informacje o liczbie wypożyczonych rowerach w zależności od dnia (dzień tygodnia, miesiąc, informacja czy był to dzień roboczy) i od pogody. Naszym zadaniem będzie predykcja sumy wypożyczonych rowerów jednego dnia w zależności od tych właśnie danych.

In [24]:
bikes = pd.read_csv('day.csv').drop(['instant', 'dteday'], axis=1)
bikes

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,1,1,12,0,4,1,2,0.254167,0.226642,0.652917,0.350133,247,1867,2114
727,1,1,12,0,5,1,2,0.253333,0.255046,0.590000,0.155471,644,2451,3095
728,1,1,12,0,6,0,2,0.253333,0.242400,0.752917,0.124383,159,1182,1341
729,1,1,12,0,0,0,1,0.255833,0.231700,0.483333,0.350754,364,1432,1796


Widzimy, że kolumna cnt jest sumą kolumn casual i registered, więc należy jeszcze usunąć te dwie kolumny.

In [25]:
bikes = bikes.drop(['casual', 'registered'], axis=1)

## Kodowanie i podział zbiorów na zbiory treningowe i testowe

In [26]:
len(data.district.value_counts())  # Sprawdzamy liczbę różnych wartości w kolumnie district

10

Ze względu na małą liczbę różnych wartości zastosujemy OneHotEncoding.

In [27]:
from category_encoders import OneHotEncoder

encoder = OneHotEncoder(cols=['district'])
data = encoder.fit_transform(data, data['m2_price'])

X = data.drop('m2_price', axis = 1)
y = data.m2_price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [28]:
X_2 = bikes.drop('cnt', axis=1)
y_2 = bikes.cnt
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.3, random_state=42)

## Dopasowanie SVM do zbiorów danych

In [29]:
svm = SVR()

svm.fit(X_train, y_train)

print(f'RMSE: {mean_squared_error(y_test, svm.predict(X_test), squared=False)}')

RMSE: 921.4746771019635


In [30]:
svm_2 = SVR()

svm_2.fit(X_train_2, y_train_2)

print(f'RMSE: {mean_squared_error(y_test_2, svm_2.predict(X_test_2), squared=False)}')

RMSE: 1989.4110287085273


Otrzymaliśmy wstępne modele z widocznymi wartościami RMSE otrzymanymi na zbiorach testowych, dalej będziemy starali się polepszyć te wyniki.

## Skalowanie danych

Ponieważ ramka danych bikes miała w większości kolumn przeskalowane dane już przy jej wczytywaniu przetestujemy zasadność skalowania jedynie na danych ze zbioru apartments.

In [31]:
scaler = MinMaxScaler()
data[['construction_year', 'surface', 'floor', 'no_rooms']] = scaler.fit_transform(data[[
    'construction_year', 'surface', 'floor', 'no_rooms'
]])

X = data.drop('m2_price', axis = 1)
y = data.m2_price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [32]:
svm = SVR()

svm.fit(X_train, y_train)

print(f'RMSE: {mean_squared_error(y_test, svm.predict(X_test), squared=False)}')

RMSE: 901.0384096140684


Rzeczywiście po przeskalowaniu danych MinMaxScalerem uzyskujemy niższy RMSE względem poprzedniego (>921). To pokazuje, że warto wcześniej skalować dane.

## Random Search

In [33]:
parameters = {'C': np.linspace(0.1, 2000, 2000),
              'gamma': ['scale', 'auto', 1],
              'degree': np.arange(1, 30, 1)}

svm_rand = RandomizedSearchCV(svm, parameters, cv=3, n_iter=200)

svm_rand.fit(X_train, y_train)
svm_rand.best_params_

{'gamma': 'scale', 'degree': 13, 'C': 1951.9783891945972}

In [34]:
best_estimator = svm_rand.best_estimator_
print(f'RMSE: {mean_squared_error(y_test, best_estimator.predict(X_test), squared=False)}')

RMSE: 154.99495482468785


Strojenie parametrów znacząco pomogło poprawić model, wartość RMSE bardzo spadła.
Sprawdzimy jeszcze wartość RMSE, aby się upewnić, że nie mamy do czynienia z overfittingiem.

In [39]:
print(f'RMSE (train): {mean_squared_error(y_train, best_estimator.predict(X_train), squared=False)}')

RMSE (train set): 136.58094403428802


Teraz znajdziemy lepsze hiperparametry dla drugiego modelu.

In [36]:
parameters = {'C': np.linspace(0.1, 2000, 2000),
              'gamma': ['scale', 'auto', 1],
              'degree': np.arange(1, 30, 1)}

svm_rand_2 = RandomizedSearchCV(svm_2, parameters, cv=3, n_iter=200)

svm_rand_2.fit(X_train_2, y_train_2)
svm_rand_2.best_params_

{'gamma': 'auto', 'degree': 16, 'C': 1989.9954977488742}

In [40]:
best_estimator_2 = svm_rand_2.best_estimator_
print(f'RMSE: {mean_squared_error(y_test_2, best_estimator_2.predict(X_test_2), squared=False)}')
print(f'RMSE (train): {mean_squared_error(y_train_2, best_estimator_2.predict(X_train_2), squared=False)}')

RMSE: 739.8409416809249
RMSE (train): 749.2615194606091


Widzimy, że strojenie hiperparametrów bardzo pomogło osiągać lepsze wyniki.